# Google Spanner

> [Spanner](https://cloud.google.com/spanner) is a highly scalable database that combines unlimited scalability with relational semantics, such as secondary indexes, strong consistency, schemas, and SQL providing 99.999% availability in one easy solution.

This notebook goes over how to use `Spanner Graph` for GraphRAG with the custom retriever `SpannerGraphVectorContextRetriever` and compares the response of GraphRAG with conventional RAG.

Learn more about the package on [GitHub](https://github.com/googleapis/langchain-google-spanner-python/).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-spanner-python/blob/main/docs/graph_rag.ipynb)

## Before You Begin

To run this notebook, you will need to do the following:

 * [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
 * [Enable the Cloud Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com)
 * [Create a Spanner instance](https://cloud.google.com/spanner/docs/create-manage-instances)
 * [Create a Spanner database](https://cloud.google.com/spanner/docs/create-manage-databases)

### 🦜🔗 Library Installation
The integration lives in its own `langchain-google-spanner` package, so we need to install it.

In [ ]:
%pip install --upgrade --quiet langchain-google-spanner langchain-google-vertexai langchain-experimental spanner-graph-notebook

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [2]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### 🔐 Authentication
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

* If you are using Colab to run this notebook, use the cell below and continue.
* If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [1]:
from google.colab import auth

auth.authenticate_user()

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [2]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}
%env GOOGLE_CLOUD_PROJECT={PROJECT_ID}

Updated property [core/project].
env: GOOGLE_CLOUD_PROJECT=google.com:cloud-spanner-demo


### 💡 API Enablement
The `langchain-google-spanner` package requires that you [enable the Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com) in your Google Cloud Project.

In [3]:
# enable Spanner API
!gcloud services enable spanner.googleapis.com

## Usage

### Set Spanner database values
Find your database values, in the [Spanner Instances page](https://console.cloud.google.com/spanner?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687).

In [4]:
# @title Set Your Values Here { display-mode: "form" }
INSTANCE = "my-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
GRAPH_NAME = "my_graph"  # @param {type: "string"}

### SpannerGraphStore

To initialize the `SpannerGraphStore` class you need to provide 3 required arguments and other arguments are optional and only need to pass if it's different from default ones

1.   a Spanner instance id;
2.   a Spanner database id belongs to the above instance id;
3.   a Spanner graph name used to create a graph in the above database.

In [5]:
from langchain_google_spanner import SpannerGraphStore

graph_store = SpannerGraphStore(
    instance_id=INSTANCE,
    database_id=DATABASE,
    graph_name=GRAPH_NAME,
)

#### Add Graph Documents
To add graph documents in the graph store.

In [34]:
# @title Load Documents
from langchain_core.documents import Document

text_snippets_category_drone = [
    # Product 1
    """
  Product: SkyHawk Zephyr Drone
  Price: $129.99
  Weight: 220g (7.8 oz)
  Take Flight with Ease:
  The SkyHawk Zephyr is the perfect drone for beginners. Designed with simplicity and stability in mind, it offers an intuitive flying experience right out of the box.
  Features:
  User-Friendly Controls: Effortlessly maneuver with easy-to-use controls and automatic takeoff and landing.
  Stable Flight: Equipped with advanced stabilization technology for smooth and steady flights, even in light wind.
  Durable Design: Crafted from high-quality materials to withstand minor bumps and crashes.
  HD Camera: Capture stunning aerial photos and videos with the built-in high-definition camera.
  Long Flight Time: Enjoy up to 15 minutes of flight time on a single charge.
  Headless Mode: Fly in any direction without worrying about the drone's orientation.
  One-Key Return: Simply press a button to bring the drone back to its takeoff point.
  Start your aerial adventure today with the SkyHawk Zephyr!
""",
    # Bundle 1
    """
  Bundle:SkyHawk Zephyr Starter Package
  Price: $129.99
  Everything you need to begin your drone journey:
  This package includes the essentials to get you flying and capturing stunning aerial footage. Perfect for first-time pilots who want a user-friendly and reliable drone.
  Package Includes:
  SkyHawk Zephyr Drone
  Remote Controller
  Rechargeable Battery
  USB Charging Cable
  Spare Propellers
  User Manual
  Take flight with the SkyHawk Zephyr Starter Package!
""",
    # Bundle 2
    """
  Bunlde: SkyHawk Zephyr Explorer Package
  Price: $179.99
  Enhance your flight experience with added value and accessories:
  This package is ideal for pilots who want to extend their flight time and enhance their drone's safety. It includes valuable accessories that elevate your aerial adventures.
  Package Includes:
  SkyHawk Zephyr Drone
  Remote Controller
  Rechargeable Battery + Extended Battery
  USB Charging Cable
  Spare Propellers
  Propeller Guards
  User Manual
  Carrying Case
  Explore new heights with the SkyHawk Zephyr Explorer Package!
""",
    # Accessory 1
    """
  Product: SkyHawk Zephyr Extended Battery
  Price: $29.99
  Weight: 70g (2.5 oz)
  Fly Longer, Explore More:
  Don't let battery life limit your aerial explorations. This extended battery for the SkyHawk Zephyr drone provides significantly more flight time, allowing you to capture more breathtaking footage and explore further.
  Features:
  Increased Capacity: Enjoy up to 25 minutes of flight time on a single charge, a 67% increase over the standard battery.
  Easy Installation: Seamlessly swap out with the original battery in seconds.
  Safe and Reliable: Built to the same high-quality standards as the SkyHawk Zephyr drone.
  Extend your flight time and capture more memories with the SkyHawk Zephyr Extended Battery!
  Accessory Of:
  SkyHawk Zephyr Drone
""",
    # Accessory 2
    """
  Product: SkyHawk Zephyr Propeller Guards
  Price: $14.99
  Weight: 20g (0.7 oz)
  Fly with Confidence:
  Protect your SkyHawk Zephyr drone and those around you with these lightweight and durable propeller guards.
  Features:
  Enhanced Safety: Prevent accidental damage to the propellers and surroundings during flight.
  Lightweight Design: Minimal impact on flight performance.
  Easy Installation: Snap on and off in seconds for quick and convenient use.
  Increased Durability: Provides an extra layer of protection for your drone.
  Fly worry-free with the SkyHawk Zephyr Propeller Guards!
  Accessory Of:
  SkyHawk Zephyr Drone
""",
    # Product 2 (upgrade of Product 1)
    """
  Product: SkyHawk Zephyr 2.0 - Reach New Heights
  Price: $199.99
  Weight: 250g (8.8 oz)
  Soar to New Heights:

  This is an upgrade from SkyHawk Zephyr Drone
  The SkyHawk Zephyr 2.0 builds upon the success of its predecessor, offering enhanced features and improved performance for an even more incredible flying experience.
  Features:
  4K Camera: Capture breathtaking aerial footage in stunning 4K resolution with enhanced image stabilization.
  Extended Flight Time: Enjoy up to 20 minutes of flight time on a single charge.
  Increased Range: Fly farther with an extended control range of up to 1.2km (3/4 mile).
  Intelligent Flight Modes: Explore creative filming options with Follow Me, Orbit, and Waypoint modes.
  GPS-Assisted Flight: Benefit from precise positioning and enhanced safety features like Return-to-Home.
  Upgraded Motors: Experience smoother, more responsive flight and increased agility.
  Streamlined Design: A sleek and aerodynamic design for improved flight performance.
  Upgrade your aerial photography and videography with the SkyHawk Zephyr 2.0!
""",
    # Deal 1
    """
  Limited-Time Offer: SkyHawk Zephyr Propeller Guards
  Fly with Confidence & Save!
  Deal Price: $9.99  (Original Price: $14.99)
  You Save: 33%
  Offer valid till 31st January 2025 or while supplies last.
""",
]

text_snippets_category_camera = [
    # Product 1
    """
  Product: SnapShot Voyager Camera 3.0
  Price: $249.99
  Weight: 350g (12.3 oz)
  Capture Your World with Clarity:
  The SnapShot Voyager is the ideal camera for both aspiring photographers and casual users. Designed for ease of use and exceptional image quality, it allows you to capture your memories with stunning detail.
  Features:
  Intuitive Interface: Navigate settings and modes effortlessly with a user-friendly touchscreen and simple controls.
  High-Resolution Sensor: Capture sharp, vibrant photos and videos with a 24-megapixel sensor.
  Optical Image Stabilization: Reduce blur from camera shake for clear images even in low light or while in motion.
  4K Video Recording: Record smooth, high-definition video at 4K resolution.
  Built-in Wi-Fi: Easily transfer photos and videos to your smartphone or tablet for sharing.
  Versatile Shooting Modes: Choose from a variety of modes, including portrait, landscape, sports, and night mode.
  Long Battery Life: Enjoy extended shooting sessions with a long-lasting rechargeable battery.
  Embark on your photographic journey with the SnapShot Voyager!
""",
    # Bundle 1
    """
  Bundle: SnapShot Voyager Photography Essentials
  Price: $299.99
  Everything you need to start capturing stunning images:
  This package includes the essentials to get you started with photography. Perfect for beginners and enthusiasts who want a versatile and easy-to-use camera with the necessary accessories.
  Package Includes:
  SnapShot Voyager Camera 3.0
  16GB SD Memory Card
  Camera Carrying Case
  Rechargeable Battery
  USB Charging Cable
  Lens Cleaning Cloth
  User Manual
  Unleash your creative potential with the SnapShot Voyager Photography Essentials!
""",
    # Bundle 2
    """
  Bundle: SnapShot Voyager Creative Studio Kit
  Price: $349.99
  Everything you need to expand your photographic creativity:
  This package includes accessories to help you take your photography to the next level. Perfect for enthusiasts who want to experiment with different techniques and capture more professional-looking images.
  Package Includes:
  SnapShot Voyager Camera 3.0
  32GB SD Memory Card
  Mini Tripod
  UV Protection Filter
  Polarizing Filter
  Camera Bag
  Rechargeable Battery
  USB Charging Cable
  Lens Cleaning Kit
  User Manual
  Unlock your artistic vision with the SnapShot Voyager Creative Studio Kit!
""",
    # Accessory 1
    """
  Product: SnapShot Voyager Lens Hood
  Price: $19.99
  Weight: 30g (1.1 oz)
  Capture Images with Enhanced Clarity:
  Reduce glare and lens flare with the SnapShot Voyager Lens Hood. This lightweight and durable accessory helps you achieve better contrast and richer colors in your photos.
  Features:
  Improved Image Quality: Minimizes unwanted light from entering the lens, reducing glare and lens flare.
  Lens Protection: Provides added protection to the front of your lens from accidental bumps and scratches.
  Durable Construction: Made from high-quality materials for long-lasting use.
  Easy Attachment: Screws securely onto the lens for a snug and reliable fit.
  Enhance your photography with the SnapShot Voyager Lens Hood!
  Accessory Of:
  SnapShot Voyager Camera 3.0
""",
    # Accessory 2
    """
  Product: SnapShot Voyager Remote Shutter Release
  Price: $24.99
  Weight: 40g (1.4 oz)
  Capture Blur-Free Images with Ease:
  Eliminate camera shake and capture sharp, clear photos, especially in low-light conditions or when using slow shutter speeds, with the SnapShot Voyager Remote Shutter Release.
  Features:
  Reduced Camera Shake: Allows you to trigger the shutter without touching the camera, minimizing movement and ensuring sharper images.
  Convenient Control: Provides a comfortable and ergonomic grip with a simple button to activate the shutter.
  Long Cable: Offers flexibility and ease of use with a generous cable length.
  Easy Connection: Plugs directly into the camera's remote port for quick and simple setup.
  Capture perfect moments with the SnapShot Voyager Remote Shutter Release!
  Accessory Of:
  SnapShot Voyager Camera
""",
    # Deal 1
    """
  Limited-Time Offer: SnapShot Voyager Remote Shutter Release
  Capture Blur-Free Images & Save!
  Deal Price: $16.99 (Original Price: $24.99)
  You Save: 32%
  Offer valid till 28th February 2025 or while supplies last.
""",
]

documents_category_drone = [
    Document(page_content=t) for t in text_snippets_category_drone
]
documents_category_camera = [
    Document(page_content=t) for t in text_snippets_category_camera
]

In [ ]:
# @title Extract Nodes and Edges
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings


def print_graph(graph_documents):
    for doc in graph_documents:
        print(doc.source.page_content[:100])
        print(doc.nodes)
        print(doc.relationships)
        print()


llm = ChatVertexAI(model="gemini-1.5-flash", temperature=0)
llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Product", "Bundle", "Deal"],
    allowed_relationships=[
        "IsBundledAs",
        "IsAccessoryOf",
        "IsUpgradeOf",
        "DealForProduct",
    ],
    node_properties=[
        "name",
        "price",
        "weight",
        "deal_end_date",
        "features",
    ],
)

graph_documents = []
graph_documents.extend(
    llm_transformer.convert_to_graph_documents(documents_category_drone)
)
graph_documents.extend(
    llm_transformer.convert_to_graph_documents(documents_category_camera)
)

# Add embeddings to the graph documents for Product nodes
embedding_service = VertexAIEmbeddings(model_name="text-embedding-004")
for graph_document in graph_documents:
    for node in graph_document.nodes:
        if "features" in node.properties:
            node.properties["embedding"] = embedding_service.embed_query(
                node.properties["features"]
            )

print_graph(graph_documents)

In [ ]:
# @title Post process extracted nodes and edges

# set of all valid products
products = set()


def prune_invalid_products():
    for graph_document in graph_documents:
        nodes_to_remove = []
        relationships_to_remove = []
        for node in graph_document.nodes:
            if node.type == "Product" and "features" not in node.properties:
                nodes_to_remove.append(node)
            else:
                products.add(node.id)
        for node in nodes_to_remove:
            graph_document.nodes.remove(node)


def is_not_a_listed_product(node):
    if node.type == "Product" and node.id not in products:
        return True
    return False


def prune_dangling_relationships():
    # now remove all dangling relationships
    for graph_document in graph_documents:
        relationships_to_remove = []
        for relationship in graph_document.relationships:
            if is_not_a_listed_product(relationship.source) or is_not_a_listed_product(
                relationship.target
            ):
                relationships_to_remove.append(relationship)
        for relationship in relationships_to_remove:
            graph_document.relationships.remove(relationship)


prune_invalid_products()
prune_dangling_relationships()
print_graph(graph_documents)

In [ ]:
# @title Load data to Spanner Graph
# Uncomment the line below, if you want to cleanup from
# previous iterations.
# BeWARE - THIS COULD REMOVE DATA FROM YOUR DATABASE !!!
graph_store.cleanup()
graph_store.add_graph_documents(graph_documents)

### Visualization

In [37]:
# @title Schema
%%spanner_graph --project {PROJECT_ID} --instance {INSTANCE} --database {DATABASE}
GRAPH my_graph #Change this to your graph name
MATCH p = ()->()
RETURN TO_JSON(p) AS path_json


## GraphRAG flow using Spanner Graph

In [70]:
# @title Question Answering Prompt
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate

from IPython.display import Markdown
import textwrap


# Retrieve and generate using the relevant snippets of the blog.
def format_docs(docs):
    print("Context Retrieved: \n")
    for doc in docs:
        print(textwrap.fill(doc.page_content, width=80))
        print("\n")

    context = "\n\n".join(doc.page_content for doc in docs)
    return context


SPANNERGRAPH_QA_TEMPLATE = """
You are a helpful and friendly AI assistant for question answering tasks.
Create a human readable answer for the for the question.
You should only use the information provided in the context and not use your internal knowledge.
Don't add any information.
Here is an example:

Question: Which funds own assets over 10M?
Context:[name:ABC Fund, name:Star fund]"
Helpful Answer: ABC Fund and Star fund have assets over 10M.

Follow this example when generating answers.
You are given the following information:
- `Question`: the natural language question from the user
- `Graph Schema`: contains the schema of the graph database
- `Graph Query`: A Spanner Graph GQL query equivalent of the question from the user used to extract context from the graph database
- `Context`: The response from the graph database as context
Information:
Question: {question}
Graph Schema: {graph_schema}
Context: {context}

Format your answer to be easily parsable by a human.
Use lists and  bullet points where applicable.
Helpful Answer:"""

prompt = PromptTemplate(
    template=SPANNERGRAPH_QA_TEMPLATE,
    input_variables=["question", "graph_schema", "context"],
)


chain = prompt | llm | StrOutputParser()

In [71]:
# @title RAG using Vector Search and Graph Expansion
import textwrap
from langchain_google_spanner import SpannerGraphVectorContextRetriever
from langchain_google_vertexai import VertexAIEmbeddings


def use_node_vector_retriever(
    question, graph_store, embedding_service, label_expr, expand_by_hops
):
    retriever = SpannerGraphVectorContextRetriever.from_params(
        graph_store=graph_store,
        embedding_service=embedding_service,
        label_expr=label_expr,
        expand_by_hops=expand_by_hops,
        top_k=1,
        k=10,
    )
    context = format_docs(retriever.invoke(question))
    return context


embedding_service = VertexAIEmbeddings(model_name="text-embedding-004")
question = "I am looking for an beginner drone. Please give me some recommendations."
context = use_node_vector_retriever(
    question, graph_store, embedding_service, label_expr="Product", expand_by_hops=1
)

answer = chain.invoke(
    {"question": question, "graph_schema": graph_store.get_schema, "context": context}
)

print("\n\nAnswer:\n")
print(textwrap.fill(answer, width=80))

Context Retrieved: 

[{"kind": "node", "labels": ["Product"], "properties": {"features": "User-
Friendly Controls, Stable Flight, Durable Design, HD Camera, Long Flight Time,
Headless Mode, One-Key Return", "id": "Skyhawk Zephyr Drone", "name": "SkyHawk
Zephyr Drone", "price": "$129.99", "weight": "220g (7.8 oz)"}}, {"kind": "edge",
"labels": ["Bundle_ISBUNDLEDAS_Product"], "properties": {"id": "Skyhawk Zephyr
Explorer Package", "target_id": "Skyhawk Zephyr Drone"}}, {"kind": "node",
"labels": ["Bundle"], "properties": {"id": "Skyhawk Zephyr Explorer Package",
"name": "SkyHawk Zephyr Explorer Package", "price": "$179.99"}}]


[{"kind": "node", "labels": ["Product"], "properties": {"features": "User-
Friendly Controls, Stable Flight, Durable Design, HD Camera, Long Flight Time,
Headless Mode, One-Key Return", "id": "Skyhawk Zephyr Drone", "name": "SkyHawk
Zephyr Drone", "price": "$129.99", "weight": "220g (7.8 oz)"}}, {"kind": "edge",
"labels": ["Bundle_ISBUNDLEDAS_Product"], "propertie

## Compare with Conventional RAG

In [59]:
# @title Define table name
TABLE_NAME = "my_table"  # @param {"type":"string"}

In [67]:
# @title Setup and load data for vector search
from langchain_google_spanner import SpannerVectorStore
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

import uuid


def load_data_for_vector_search(splits):
    embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

    db = SpannerVectorStore(
        instance_id=INSTANCE,
        database_id=DATABASE,
        table_name=TABLE_NAME,
        embedding_service=embeddings,
    )
    # Add the chunks to Spanner Vector Store with embeddings
    ids = [str(uuid.uuid4()) for _ in range(len(splits))]
    row_ids = db.add_documents(splits, ids)


# Create splits for documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
splits = text_splitter.split_documents(
    documents_category_drone + documents_category_camera
)

# Initialize table and load data
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")
load_data_for_vector_search(splits)

In [72]:
# @title Create a conventional RAG chain

from langchain_core.runnables import RunnablePassthrough
from langchain_google_spanner import SpannerVectorStore
import textwrap


# Retrieve and generate using the relevant snippets of the blog.
def format_docs(docs):
    print("Context Retrieved: \n")
    for doc in docs:
        print(textwrap.fill(doc.page_content, width=80))
        print("\n")

    context = "\n\n".join(doc.page_content for doc in docs)
    return context


prompt = PromptTemplate(
    template="""
    You are a friendly digital shopping assistant.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Question: {question}
    Context: {context}
    Answer:
  """,
    input_variables=["context", "question"],
)

# Create a rag chain
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

db = SpannerVectorStore(
    instance_id=INSTANCE,
    database_id=DATABASE,
    table_name=TABLE_NAME,
    embedding_service=embeddings,
)
vector_retriever = db.as_retriever(search_kwargs={"k": 3})
rag_chain = (
    {
        "context": vector_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [69]:
# @title Run conventional RAG chain
import textwrap

question = "I am looking for an beginner drone. Please give me some recommendations"
resp = rag_chain.invoke(question)
print("\n\nRag Response:\n")
print(textwrap.fill(resp, width=80))

Context Retrieved: 

Product: SkyHawk Zephyr Drone   Price: $129.99   Weight: 220g (7.8 oz)   Take
Flight with Ease:   The SkyHawk Zephyr is the perfect drone for beginners.
Designed with simplicity and stability in mind, it offers an intuitive flying
experience right out of the box.   Features:   User-Friendly Controls:
Effortlessly maneuver with easy-to-use controls and automatic takeoff and
landing.   Stable Flight: Equipped with advanced stabilization technology for
smooth and steady flights, even in light wind.   Durable Design: Crafted from
high-quality materials to withstand minor bumps and crashes.   HD Camera:
Capture stunning aerial photos and videos with the built-in high-definition
camera.   Long Flight Time: Enjoy up to 15 minutes of flight time on a single
charge.   Headless Mode: Fly in any direction without worrying about the drone's
orientation.   One-Key Return: Simply press a button to bring the drone back to
its takeoff point.   Start your aerial adventure today wit

## Clean up the graph

> USE IT WITH CAUTION!

Clean up all the nodes/edges in your graph and delete your graph definition.

In [ ]:
graph_store.cleanup()